<a href="https://colab.research.google.com/github/Fatin817/CODSOFT/blob/main/TEXT_GENERATION_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Setup and libraries**

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0
time: 2.79 ms (started: 2024-06-22 16:33:08 +00:00)


In [ ]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 9.36 s (started: 2024-06-22 16:33:09 +00:00)


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense,SimpleRNN,Embedding,Dropout
from google.colab import drive
import keras
import re
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

time: 1.14 ms (started: 2024-06-22 16:54:04 +00:00)


In [ ]:
drive.mount('/content/drive')

In [ ]:
from typing_extensions import Concatenate
# Load the NPZ file (/content/drive/MyDrive/deepwriting_dataset/deepwriting_validation.npz)
data1 = np.load('/content/drive/MyDrive/deepwriting_dataset/deepwriting_training.npz', allow_pickle=True)
data2 = np.load('/content/drive/MyDrive/deepwriting_dataset/deepwriting_validation.npz', allow_pickle=True)
data1, data2

(NpzFile '/content/drive/MyDrive/deepwriting_dataset/deepwriting_training.npz' with keys: subject_labels, eoc_labels, alphabet, strokes, eow_labels...,
 NpzFile '/content/drive/MyDrive/deepwriting_dataset/deepwriting_validation.npz' with keys: subject_labels, eoc_labels, alphabet, strokes, eow_labels...)

time: 23.5 ms (started: 2024-06-22 16:33:18 +00:00)


## **Training**

In [ ]:
# Shape of the training data
subject_labels_train = data1['subject_labels']
eoc_labels_train = data1['eoc_labels']
alphabet_train = data1['alphabet']
strokes_train = data1['strokes']
eow_labels_train = data1['eow_labels']

subject_labels_train.shape, eoc_labels_train.shape, alphabet_train.shape, strokes_train.shape, eow_labels_train.shape


((34577,), (34577,), (70,), (34577,), (34577,))

time: 5.91 s (started: 2024-06-22 16:33:18 +00:00)


In [ ]:
subject_labels_train.dtype, eoc_labels_train.dtype, alphabet_train.dtype, strokes_train.dtype, eow_labels_train.dtype

(dtype('int64'), dtype('O'), dtype('<U1'), dtype('O'), dtype('O'))

time: 6.99 ms (started: 2024-06-22 16:33:28 +00:00)


In [ ]:
# Alphabet datatype is a unicode string
text_train = ''.join(alphabet_train)
text_train

"0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'.,-()/"

time: 8.62 ms (started: 2024-06-22 16:33:29 +00:00)


In [ ]:
# Create vocabulary from text
vocabulary_train = sorted(set(text_train))
char_to_index_train = {char: index for index, char in enumerate(vocabulary_train)}
index_to_char_train = np.array(vocabulary_train)

time: 3.01 ms (started: 2024-06-22 16:33:34 +00:00)


In [ ]:
char_to_index_train

{"'": 0,
 '(': 1,
 ')': 2,
 ',': 3,
 '-': 4,
 '.': 5,
 '/': 6,
 '0': 7,
 '1': 8,
 '2': 9,
 '3': 10,
 '4': 11,
 '5': 12,
 '6': 13,
 '7': 14,
 '8': 15,
 '9': 16,
 'A': 17,
 'B': 18,
 'C': 19,
 'D': 20,
 'E': 21,
 'F': 22,
 'G': 23,
 'H': 24,
 'I': 25,
 'J': 26,
 'K': 27,
 'L': 28,
 'M': 29,
 'N': 30,
 'O': 31,
 'P': 32,
 'Q': 33,
 'R': 34,
 'S': 35,
 'T': 36,
 'U': 37,
 'V': 38,
 'W': 39,
 'X': 40,
 'Y': 41,
 'Z': 42,
 'a': 43,
 'b': 44,
 'c': 45,
 'd': 46,
 'e': 47,
 'f': 48,
 'g': 49,
 'h': 50,
 'i': 51,
 'j': 52,
 'k': 53,
 'l': 54,
 'm': 55,
 'n': 56,
 'o': 57,
 'p': 58,
 'q': 59,
 'r': 60,
 's': 61,
 't': 62,
 'u': 63,
 'v': 64,
 'w': 65,
 'x': 66,
 'y': 67,
 'z': 68}

time: 6.3 ms (started: 2024-06-22 16:33:56 +00:00)


In [ ]:
index_to_char_train

array(["'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I',
       'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
       'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z'], dtype='<U1')

time: 4.67 ms (started: 2024-06-22 16:34:00 +00:00)


In [ ]:
# Convert text to integer
text_as_int_train = np.array([char_to_index_train[char] for char in text_train])
text_as_int_train

array([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 43, 44, 45, 46, 47, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
       32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,  0,  5,  3,  4,  1,  2,
        6])

time: 4.63 ms (started: 2024-06-22 16:34:04 +00:00)


In [ ]:
#Sequences
sequences_length = 100
char_dataset_train = tf.data.Dataset.from_tensor_slices(text_as_int_train)
sequences_train = char_dataset_train.batch(sequences_length + 1)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset_train = sequences_train.map(split_input_target)


time: 28.5 ms (started: 2024-06-22 16:34:23 +00:00)


In [ ]:
BATCH_SIZE = 1
BUFFER_SIZE = 10000

dataset_train = dataset_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


time: 7.71 ms (started: 2024-06-22 16:34:26 +00:00)


## **Validation**

In [ ]:
# Shape of the training data
subject_labels_val = data2['subject_labels']
eoc_labels_val = data2['eoc_labels']
alphabet_val = data2['alphabet']
strokes_val = data2['strokes']
eow_labels_val = data2['eow_labels']

subject_labels_val.shape, eoc_labels_val.shape, alphabet_val.shape, strokes_val.shape, eow_labels_val.shape


((705,), (705,), (70,), (705,), (705,))

time: 135 ms (started: 2024-06-22 16:34:31 +00:00)


In [ ]:
subject_labels_val.dtype, eoc_labels_val.dtype, alphabet_val.dtype, strokes_val.dtype, eow_labels_val.dtype

(dtype('int64'), dtype('O'), dtype('<U1'), dtype('O'), dtype('O'))

time: 4.24 ms (started: 2024-06-22 16:34:33 +00:00)


In [ ]:
# Alphabet datatype is a unicode string
text_val = ''.join(alphabet_val)
text_val

"0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'.,-()/"

time: 6.11 ms (started: 2024-06-22 16:34:34 +00:00)


In [ ]:
# Create vocabulary from text
vocabulary_val = sorted(set(text_val))
char_to_index_val = {char: index for index, char in enumerate(vocabulary_val)}
index_to_char_val = np.array(vocabulary_val)

time: 983 µs (started: 2024-06-22 16:34:36 +00:00)


In [ ]:
char_to_index_val

{"'": 0,
 '(': 1,
 ')': 2,
 ',': 3,
 '-': 4,
 '.': 5,
 '/': 6,
 '0': 7,
 '1': 8,
 '2': 9,
 '3': 10,
 '4': 11,
 '5': 12,
 '6': 13,
 '7': 14,
 '8': 15,
 '9': 16,
 'A': 17,
 'B': 18,
 'C': 19,
 'D': 20,
 'E': 21,
 'F': 22,
 'G': 23,
 'H': 24,
 'I': 25,
 'J': 26,
 'K': 27,
 'L': 28,
 'M': 29,
 'N': 30,
 'O': 31,
 'P': 32,
 'Q': 33,
 'R': 34,
 'S': 35,
 'T': 36,
 'U': 37,
 'V': 38,
 'W': 39,
 'X': 40,
 'Y': 41,
 'Z': 42,
 'a': 43,
 'b': 44,
 'c': 45,
 'd': 46,
 'e': 47,
 'f': 48,
 'g': 49,
 'h': 50,
 'i': 51,
 'j': 52,
 'k': 53,
 'l': 54,
 'm': 55,
 'n': 56,
 'o': 57,
 'p': 58,
 'q': 59,
 'r': 60,
 's': 61,
 't': 62,
 'u': 63,
 'v': 64,
 'w': 65,
 'x': 66,
 'y': 67,
 'z': 68}

time: 8.42 ms (started: 2024-06-22 16:34:39 +00:00)


In [ ]:
index_to_char_val

array(["'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I',
       'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
       'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z'], dtype='<U1')

time: 4.2 ms (started: 2024-06-22 16:34:48 +00:00)


In [ ]:
# Convert text to integer
text_as_int_val = np.array([char_to_index_val[char] for char in text_val])
text_as_int_val

array([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 43, 44, 45, 46, 47, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
       32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,  0,  5,  3,  4,  1,  2,
        6])

time: 4.51 ms (started: 2024-06-22 16:34:50 +00:00)


In [ ]:
#Sequences
sequences_length = 100
char_dataset_val = tf.data.Dataset.from_tensor_slices(text_as_int_val)
sequences_val = char_dataset_val.batch(sequences_length + 1)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset_val = sequences_val.map(split_input_target)


time: 28.6 ms (started: 2024-06-22 16:35:03 +00:00)


In [ ]:
BATCH_SIZE = 1
BUFFER_SIZE = 10000

dataset_val = dataset_val.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


time: 9.78 ms (started: 2024-06-22 16:35:08 +00:00)


In [ ]:
dataset_val, dataset_train

(<_BatchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>,
 <_BatchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>)

time: 4.25 ms (started: 2024-06-22 16:37:32 +00:00)


In [ ]:
model = tf.keras.Sequential([
    Embedding(len(vocabulary_train), 256, batch_input_shape=[BATCH_SIZE, None]),
    SimpleRNN(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform', activation='relu'),
    SimpleRNN(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform', activation='relu'),
    Dropout(0.5),
    Dense(len(vocabulary_train),activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (1, None, 256)            17664     
                                                                 
 simple_rnn_20 (SimpleRNN)   (1, None, 1024)           1311744   
                                                                 
 simple_rnn_21 (SimpleRNN)   (1, None, 1024)           2098176   
                                                                 
 dropout (Dropout)           (1, None, 1024)           0         
                                                                 
 dense_18 (Dense)            (1, None, 69)             70725     
                                                                 
Total params: 3498309 (13.34 MB)
Trainable params: 3498309 (13.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
time: 277 ms 

In [ ]:
history = model.fit(dataset_train, epochs=50,validation_data=dataset_val)

Epoch 1/50
1/1 [==============================] - 4s 4s/step - loss: 4.2395 - accuracy: 0.0000e+00 - val_loss: 4.2232 - val_accuracy: 0.0735
Epoch 2/50
1/1 [==============================] - 1s 726ms/step - loss: 4.2267 - accuracy: 0.0000e+00 - val_loss: 4.2102 - val_accuracy: 0.0882
Epoch 3/50
1/1 [==============================] - 1s 718ms/step - loss: 4.2108 - accuracy: 0.0735 - val_loss: 4.1977 - val_accuracy: 0.2059
Epoch 4/50
1/1 [==============================] - 1s 743ms/step - loss: 4.1946 - accuracy: 0.1324 - val_loss: 4.1855 - val_accuracy: 0.3382
Epoch 5/50
1/1 [==============================] - 1s 730ms/step - loss: 4.1880 - accuracy: 0.1176 - val_loss: 4.1733 - val_accuracy: 0.4853
Epoch 6/50
1/1 [==============================] - 1s 769ms/step - loss: 4.1721 - accuracy: 0.1765 - val_loss: 4.1610 - val_accuracy: 0.6176
Epoch 7/50
1/1 [==============================] - 1s 732ms/step - loss: 4.1612 - accuracy: 0.3088 - val_loss: 4.1484 - val_accuracy: 0.7500
Epoch 8/50
1/1 

In [ ]:
import tensorflow as tf

def generate_text(model, start_string, char_to_index_val, index_to_char_val, num_generate=1000, temperature=1.0):
    # Convert start_string to indices
    input_eval = [char_to_index_val[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []  # List to store the generated text

    model.reset_states()  # Reset model states
    for i in range(num_generate):
        predictions = model(input_eval)  # Get predictions
        predictions = tf.squeeze(predictions, 0)  # Remove batch dimension

        # Apply temperature to control randomness
        predictions = predictions / temperature

        # Sample from the distribution
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Append the predicted character to the generated text
        text_generated.append(index_to_char_val[predicted_id])

        # Update input_eval to include the newly predicted character
        input_eval = tf.expand_dims(input_eval[0].numpy().tolist() + [predicted_id], 0)

    generated_text = start_string + ''.join(text_generated)
    return generated_text

# Example usage
# Assuming model, char_to_index_val, and index_to_char_val are already defined and initialized
generated_text = generate_text(model, start_string="Hello", char_to_index_val=char_to_index_val, index_to_char_val=index_to_char_val)
print("Generated Text: \n", generated_text)


Generated Text: 
 HelloLFIfBQMpfGo3BeE((xLhFlw-cr6qkeE5TO6Uexx8H'gIu1nOV77gMv8Wnsej.Mugk1li9iFPDqsUo3b3qE8kemAGRwJB8u0TS5T2e)ZTv-zVLZRkg5/jzTCDLzRaWKC1TgM)vg0h/(4A7NWDVRI'MAXGmVJH)mP1tlyy1ht1PEzPJDWY,RjTmHq9/358-)gkwtMEbstuoL-9G008.mRhrck3HvvfEHkJP33YY))I2rD38g8wK0cGsaNpLzlpPGbgfw3P(LnY0,9upjGPrQvw4H,LUhbqFa8EDWjg'u7O.iBGxTooqNKjt)1tj)7hchVSPC8Um.Qw(Q)TdA49'zZv5JaOXldOoz'VWlZhjrq/vhX8EzD1OHQhB,pyA1U6-8aCv.GREnVbzXIxwbRNW41)Pf'cln5f3t'2pQNpbXdegrJpN69x2leI/AlgVY-HpMx02dixEnQKZ'9kPFEgcgzgNhIsb0vxlbNC2WbY,l1cW1.b5aOAZYOXmBIQ-PjuYjPZkl1oBMtJ.eX31/.nILWscu.0TN4JrkZdLtPDwFg4pEF0bV)4g6M-rH9/E15yL2X9z)Aal8GuLgPzaXL6IUQ676blK'vJ1f91oXZGbMDvs'YAMlTrjXVCwZBSLA-L0pKtZwqp9L5LJ8/O'QxOdcRbO/tK-.-m,lGovM8Pn7jQAyXbfj2Dp99j-dbhIcie36Fxm5dUgY-njO5M6'3t38hhuGmOOR.kbiW9eBeqc1gQlZd0KbU)h9pBajb/-eOr5klAs.ieIYM7pxOZ63FVBWcWiw'TZMjp(.M2wEt2gwZ1.v1xLt/Pn'R-Q'2-28,ppiNhCIU-'ztqambtK2)Z8TkHfPlpBnpgm'QhMSV,D9G.-XSKlVMHnwI3NapdESI21FNbQ(-C608oaR2n)qlRE'94xa1efQwyeJgcqWFHDo.oVcK27R0J(iBjyt)uF)l42,ULy48bRDkXpk32.,oIy